In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from iprocessor import add_day_name_column, add_date_name_column, smooth_sundays_rolling_w7_l

#### the python script iprocessor, smoothes the data by implementing moving average

## there is need for 
- parameters
- initialization for the compartments

In [2]:
# Sample parameters,
contacts = 2.0
transmission_prob = 0.3649
reducing_transmission = 0.764
exposed_period = 5.2  #
asymptomatic_period = 7
infectious_period = 3.7
isolated_period = 11  # 11,23
prob_asymptomatic = 0.2
prob_quarant_inf = 0.05
test_asy = 0.171
dev_symp = 0.125
mortality_isolated = 0.002
mortality_infected = 0.01

In [3]:
total_population = 82_000_000  # Total number of individuals
E0 = 2026.25
A0 = 3798
I0 = 376.4
F0 = 2255
R0 = 170204
D0 = 9060
S0 = total_population - E0 - A0 - I0 - F0 - R0 - D0
initial_conditions = [S0, E0, A0, I0, F0, R0, D0]

### Dataframe

In [4]:
df = pd.read_csv(r'German_case_period_may_aug.csv')

In [5]:
#Convert 'Date' column to datetime format----------------------------------------------------------------------
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Modification------------------------------------------------------------------------------
# Add the 'days' column
df = add_day_name_column(df)
df = add_date_name_column(df)
# second modification with w7_l-----------------------------------------------------------------------------
df_observed = smooth_sundays_rolling_w7_l(df)
df_observed
# -----------------------------------------------------------------------------------------------------

,Date,Confirmed,Deaths,Recovered,n_confirmed,n_death,n_recovered,Infection_case,date_name,days,rolling_mean_r,rolling_mean_c,rolling_mean_d
0,2020-05-01,166468,8602,157866,428.333333,16.000000,412.333333,915,Friday,1,412.333333,428.333333,16.000000
1,2020-05-02,167160,8626,158534,529.250000,22.750000,506.500000,846,Saturday,2,506.500000,529.250000,22.750000
2,2020-05-03,167753,8650,159103,602.600000,25.800000,576.800000,805,Sunday,3,576.800000,602.600000,25.800000
3,2020-05-04,168585,8693,159892,654.666667,25.833333,628.833333,607,Monday,4,628.833333,654.666667,25.833333
4,2020-05-05,169481,8731,160750,682.000000,27.285714,654.714286,447,Tuesday,5,654.714286,682.000000,27.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2020-07-28,210338,9412,200926,679.285714,4.285714,675.000000,563,Tuesday,89,675.000000,679.285714,4.285714
89,2020-07-29,211114,9418,201696,694.000000,6.000000,688.000000,919,Wednesday,90,688.000000,694.000000,6.000000
90,2020-07-30,211868,9423,202445,709.285714,5.714286,703.571429,940,Thursday,91,703.571429,709.285714,5.714286
91,2020-07-31,212712,9438,203274,749.833333,6.000000,743.833333,996,Friday,92,743.833333,749.833333,6.000000


In [6]:
# Taking 'days' time column from dataframe
t_fit_base = np.array(df_observed['days'])
tmax_base = len(t_fit_base)
print(f'{tmax_base}  time_base')
#time
t_fit_0 = np.zeros((tmax_base,2))
t_fit_0[:,0] = t_fit_base
t_fit_0[:,1] = 0
#
t_fit_1 = np.zeros((tmax_base,2))
t_fit_1[:,0] = t_fit_base
t_fit_1[:,1] = 1
#
t_fit = np.r_[t_fit_0,t_fit_1]
x= np.r_[t_fit_0,t_fit_1]
print(f'{len(t_fit)}   t_fit')
#
tmax = len(t_fit)


# for t_eval in the solve_ivp
times = np.linspace(1,tmax,tmax+100)


93  time_base
186   t_fit


### derivative equation for the flowchart

In [7]:
def derivative_rhs(x, X, contacts, transmission_prob, total_population, reducing_transmission,
                   exposed_period, asymptomatic_period, infectious_period, isolated_period,
                   prob_asymptomatic, prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected):
    S, E, A, I, F, R, D = X
    derivS = - contacts * transmission_prob * S * (I + reducing_transmission * A) / total_population
    derivE = contacts * transmission_prob * S * (I + reducing_transmission * A) / total_population - E / exposed_period
    derivA = prob_asymptomatic * E / exposed_period - A / asymptomatic_period
    derivI = (
                         1 - prob_asymptomatic) * E / exposed_period + dev_symp * A / asymptomatic_period - I / infectious_period  # +
    derivF = prob_quarant_inf * I / infectious_period - F / isolated_period + test_asy * A / asymptomatic_period  # prob_isolated_asy*A/asymptomatic_period
    derivR = (1 - prob_quarant_inf - mortality_infected) * I / infectious_period + (
                1 - mortality_isolated) * F / isolated_period + (
                         1 - dev_symp - test_asy) * A / asymptomatic_period  # (1-prob_isolated_asy)*A / asymptomatic_period
    derivD = (mortality_infected) * I / infectious_period + mortality_isolated * F / isolated_period
    return [derivS, derivE, derivA, derivI, derivF, derivR, derivD]

In [8]:
def seaifrd_model(x, contacts, initial_conditions, transmission_prob, total_population, reducing_transmission,
                  exposed_period, asymptomatic_period, infectious_period, isolated_period,
                  prob_asymptomatic, prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected):
    def derivative(x, initial_conditions):
        return derivative_rhs(x, initial_conditions, contacts, transmission_prob, total_population,
                              reducing_transmission,
                              exposed_period, asymptomatic_period, infectious_period,
                              isolated_period, prob_asymptomatic,
                              prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected)

    solution = solve_ivp(derivative, (0, tmax), initial_conditions, method='RK45',t_eval=times)
    print(solution)
    return solution  # .y.flatten()

In [9]:
def objective_function_recoverd_dead(X,isolated_period):
    t = X[:, 0]
    mode = X[:, 1]

    # Model parameters (could be passed as arguments or fixed here)
    contacts = 2
    transmission_prob = 0.3
    total_population = 82000000
    reducing_transmission = 0.55
    exposed_period = 5.2
    asymptomatic_period = 7
    infectious_period = 3.7
    #isolated_period = 12
    prob_asymptomatic = 0.34
    prob_quarant_inf = 0.9303
    test_asy = 0.271
    dev_symp = 0.125
    mortality_isolated = 0.02
    mortality_infected = 0.1

    # Solve the model
    t = t[:len(t)//2] #sorted(set(t))
    solution = seaifrd_model(t, contacts, initial_conditions, transmission_prob, total_population, reducing_transmission,
                             exposed_period, asymptomatic_period, infectious_period, isolated_period,
                             prob_asymptomatic, prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected)

    print(solution)
    time = solution.t
    recovered = solution.y[5]
    recovered = np.concatenate((recovered,recovered))
    dead = solution.y[6]
    dead = np.concatenate((dead,dead))
    retval = np.where(mode, recovered, dead)

    return retval


In [10]:
# here we have to make the length for the recovered and the dead twice!

In [11]:
data_recovered = np.ones((df_observed['n_recovered'].size,2))
data_recovered[:,0] = df_observed['n_recovered']
#
data_dead = np.zeros((df_observed['n_death'].size,2))
data_dead[:,0] = df_observed['n_death']
recovered_dead = np.r_[data_recovered,data_dead]
print(len(recovered_dead))

186


In [12]:
#recovered_dead = np.concatenate((df_observed['n_death'],df_observed['n_recovered']))
#print(len(recovered_dead))

In [13]:
### Calling curve_fit Function

In [14]:
#t_end = df_observed['days'].iloc[-1]
#print(f't_end = {(t_end)}, t_fit = {len(t_fit)}')
# Create a sequence from 0 to t_end
#t_fit = np.arange(0, 2*tmax, 1)

popt, pcov = curve_fit(objective_function_recoverd_dead, t_fit, recovered_dead)


  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  1.649e+00 ...  1.854e+02  1.860e+02]
        y: [[ 8.181e+07  8.181e+07 ...  1.071e+07  1.071e+07]
            [ 3.026e+03  3.609e+03 ...  6.586e+03  6.239e+03]
            ...
            [ 1.720e+05  1.727e+05 ...  6.520e+07  6.520e+07]
            [ 9.104e+03  9.127e+03 ...  6.063e+06  6.063e+06]]
      sol: None
 t_events: None
 y_events: None
     nfev: 392
     njev: 0
      nlu: 0
  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  1.649e+00 ...  1.854e+02  1.860e+02]
        y: [[ 8.181e+07  8.181e+07 ...  1.071e+07  1.071e+07]
            [ 3.026e+03  3.609e+03 ...  6.586e+03  6.239e+03]
            ...
            [ 1.720e+05  1.727e+05 ...  6.520e+07  6.520e+07]
            [ 9.104e+03  9.127e+03 ...  6.063e+06  6.063e+06]]
      sol: None
 t_events: None
 y_events

ValueError: operands could not be broadcast together with shapes (186,) (572,) (572,) 

In [ ]:
popt

In [ ]:
perr_r_d = np.sqrt(np.diag(pcov))
perr_r_d

In [19]:
#!python curve_fit_M.py